#### Checking and upgrading the DKube SDK

In [ ]:
import pkg_resources, sys
try:
    dkube_sdk_version = float(pkg_resources.get_distribution("dkube").version)
except:
    dkube_sdk_version = 0 ## means the dkube sdk is not installed
if dkube_sdk_version < 3.7:
    !{sys.executable} -m pip install sudo pip install  git+https://github.com/oneconvergence/dkube.git@3.7 --user >/dev/null
%reset -f

In [ ]:
import os
from time import gmtime, strftime

#### Defined by User

In [ ]:
## specify your Dkube username
DKUBEUSERNAME = "ocdkube"

## Define the model monitor name here that you will be creating 
MONITOR_NAME = "insurance-mm-sgmkr-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

PREFIX = "sagemaker/Demo-ModelMonitor"

## Dkube information
TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","")
DKUBE_URL = os.getenv("DKUBE_URL","")
DKUBE_TRAINING_CODE_NAME = "monitoring-examples"

### Specify Sagemaker details
BUCKET = os.getenv("BUCKET","")
ROLE = os.getenv("ROLE","")
REGION_NAME = os.getenv("REGION_NAME","")

ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID","")
SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY","")


### Dkube Sagemaker cluster 
SAGEMAKER_DKUBE_CLUSTER_NAME = "aws-sagemaker-cluster"

# dataset to be used as training data
DKUBE_BASE_DATASET = "insurance-data"

# the frequency with which monitoring will run
RUN_FREQUENCY = 5

if TOKEN == '' or DKUBEUSERNAME == '' or DKUBE_URL == '':
    raise TypeError("Please fill the Dkube details first (TOKEN, DKUBE_URL, DKUBEUSERNAME)")
if ACCESS_KEY=='' or SECRET_KEY=='' or ROLE=='' or REGION_NAME=='' or BUCKET=='':
    raise TypeError("Please fill the AWS_S3 details first (ACCESS_KEY, SECRET_KEY, ROLE, REGION_NAME, BUCKET)")

In [ ]:
sgmkr_config = {"MONITOR_NAME":MONITOR_NAME, "DKUBEUSERNAME":DKUBEUSERNAME,
                "TOKEN":TOKEN, "DKUBE_URL":DKUBE_URL, "ACCESS_KEY":ACCESS_KEY,
                "SECRET_KEY":SECRET_KEY, "DKUBE_BASE_DATASET":DKUBE_BASE_DATASET,
                "RUN_FREQUENCY":RUN_FREQUENCY, "DKUBE_TRAINING_CODE_NAME":DKUBE_TRAINING_CODE_NAME,
                "BUCKET":BUCKET, "PREFIX":PREFIX, "ROLE":ROLE, "REGION_NAME":REGION_NAME,
                "SAGEMAKER_DKUBE_CLUSTER_NAME":SAGEMAKER_DKUBE_CLUSTER_NAME}
%store sgmkr_config

#### Dkube Resources

In [ ]:
import time,json,shutil
from dkube.sdk import *

In [ ]:
api = DkubeApi(URL=DKUBE_URL,token=TOKEN)
if DKUBEUSERNAME == api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, first")

#### Dataset

In [ ]:
try:
    dataset = DkubeDataset(DKUBEUSERNAME, name=DKUBE_BASE_DATASET)
    dataset.update_git_details(url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")
    dataset.update_dataset_source(source="pub_url")
    api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

### Model Monitor Datasets

##### Labelled Dataset

In [ ]:
gname = MONITOR_NAME+'-groundtruth'
prefix = MONITOR_NAME+'/groundtruth'            

try:
    dataset = DkubeDataset(DKUBEUSERNAME, name=gname,remote=True)
    dataset.update_dataset_source('aws_s3')
    dataset.update_awss3_details(bucket=BUCKET,prefix=prefix,key=os.getenv("AWS_ACCESS_KEY_ID",ACCESS_KEY),secret=os.getenv("AWS_SECRET_ACCESS_KEY",SECRET_KEY))
    api.create_dataset(dataset)
    
except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    api.delete_dataset(DKUBEUSERNAME,DKUBE_BASE_DATASET,force=True)
    %store -d sgmkr_config